In [7]:
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession ,Row
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType,StructField,IntegerType
import matplotlib.pyplot as plt
from os.path import abspath
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
import os
import findspark
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


findspark.init()

In [8]:
#databricks tutorialBase
#https://docs.databricks.com/external-data/sql-server.html

In [9]:
os.environ["JAVA_HOME"] = r"/usr/lib/jvm/java-1.11.0-openjdk-amd64"
os.environ["SPARK_HOME"] = r"/home/guilherme/als_recomendacao_spark_app/spark/spark"
os.environ["LIVY_CONF_DIR"] = r"/home/guilherme/als_recomendacao_spark_app/livy_server"

In [10]:

warehouse_location = abspath('spark-warehouse')

In [11]:

conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.sql.adaptive.enabled","true")
conf.set("spark.sql.adaptive.localShuffleReader.enabled","true")
conf.set("spark.dynamicAllocation.enabled", "false")
conf.set("spark.sql.adaptive.optimizeSkewsInRebalancePartitions.enabled","true")
conf.set("spark.sql.adaptive.skewJoin.enabled","true")
conf.set("spark.sql.statistics.size.autoUpdate.enabled","true")
conf.set("spark.sql.inMemoryColumnarStorage.compressed","true")
conf.set("hive.exec.dynamic.partition", "true")
conf.set("hive.exec.dynamic.partition.mode", "nonstrict")
conf.set("spark.sql.ansi.enabled","true")
conf.set('spark.driver.extraClassPath', r"/home/guilherme/als_recomendacao_spark_app/sqlserverjars/mssql-jdbc-12.2.0.jre11.jar")
conf.set('spark.executor.extraClassPath', r"/home/guilherme/als_recomendacao_spark_app/sqlserverjars/mssql-jdbc-12.2.0.jre11.jar")
spark = SparkSession.builder\
        .config(conf=conf)\
        .config("spark.sql.warehouse.dir", warehouse_location)\
        .config("spark.sql.catalogImplementation", "hive") \
        .enableHiveSupport() \
        .getOrCreate()
    

23/07/10 00:55:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

database_host = "BIGDATA"
database_port = "1433" # update if you use a non-default port
database_name = "BIGDATA"
table = "comercial.ranking_atendimento_tim"
user = "sa"
password = "123"

url = f"jdbc:sqlserver://{database_host}:{database_port};database={database_name}"

remote_table = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

In [ ]:
remote_table